In [42]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
from shapely.ops import unary_union, cascaded_union
from shapely import wkt
from urllib.request import urlopen, HTTPError

In [2]:
import ee
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account
import geemap
import rasterio

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AY0e-g5-4Tl6bWTqUFg_b6RLUfIFdxppEmxb3s3hIyuDZFs1rWb-q_rkyWk

Successfully saved authorization token.


In [15]:
subnational_raw = gpd.read_file('Admin_2/afr_g2014_2013_2.shp')

In [16]:
subnational_raw.head()

,STATUS,DISP_AREA,ADM2_CODE,ADM2_NAME,STR2_YEAR,EXP2_YEAR,ADM1_CODE,ADM1_NAME,ADM0_CODE,ADM0_NAME,Shape_Leng,Shape_Area,FID_1,AREA,PERIMETER,CONTINENT,REGION,geometry
0,Member State,NO,40621,Ryansoro,1000,3000,40548,Gitega,43,Burundi,0.754714,0.013313,568,2.193094,8.560339,Africa,Eastern Africa,"POLYGON ((29.79610 -3.62673, 29.79516 -3.63288..."
1,Member State,NO,40756,Mutambu,1000,3000,40544,Bujumbura Rural,43,Burundi,0.549887,0.006585,568,2.193094,8.560339,Africa,Eastern Africa,"POLYGON ((29.49287 -3.51601, 29.49256 -3.51738..."
2,Member State,NO,40667,Ndava,1000,3000,40555,Mwaro,43,Burundi,0.721689,0.013685,568,2.193094,8.560339,Africa,Eastern Africa,"POLYGON ((29.82561 -3.36665, 29.82506 -3.36981..."
3,Member State,NO,40757,Nyabiraba,1000,3000,40544,Bujumbura Rural,43,Burundi,0.536116,0.006385,568,2.193094,8.560339,Africa,Eastern Africa,"POLYGON ((29.50873 -3.43328, 29.50836 -3.43353..."
4,Member State,NO,51360,Nairobi,1000,3000,51328,Nairobi,133,Kenya,1.664051,0.056580,465,47.271531,42.779034,Africa,Eastern Africa,"POLYGON ((36.90686 -1.15805, 36.90781 -1.15840..."


In [22]:
subnational = subnational_raw.drop(['DISP_AREA','EXP2_YEAR', 'CONTINENT', 'FID_1'], axis=1)

In [23]:
subnational.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6226 entries, 0 to 6225
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   STATUS      6226 non-null   object  
 1   ADM2_CODE   6226 non-null   int64   
 2   ADM2_NAME   6226 non-null   object  
 3   STR2_YEAR   6226 non-null   int64   
 4   ADM1_CODE   6226 non-null   int64   
 5   ADM1_NAME   6226 non-null   object  
 6   ADM0_CODE   6226 non-null   int64   
 7   ADM0_NAME   6226 non-null   object  
 8   Shape_Leng  6226 non-null   float64 
 9   Shape_Area  6226 non-null   float64 
 10  AREA        6226 non-null   float64 
 11  PERIMETER   6226 non-null   float64 
 12  REGION      6155 non-null   object  
 13  geometry    6226 non-null   geometry
dtypes: float64(4), geometry(1), int64(4), object(5)
memory usage: 681.1+ KB


In [24]:
subnational['geom_mapping'] = subnational['geometry'].apply(lambda x: mapping(x))
subnational['geom_list'] = subnational['geom_mapping'].apply(lambda x: list(x['coordinates']))
subnational['geom_ee'] = 1
for i in range(len(subnational)):
    if subnational.geometry[i].geom_type == 'Polygon':
        
        subnational['geom_ee'][i] = ee.Geometry.Polygon(subnational['geom_list'][i])
        
    else:
        
        subnational['geom_ee'][i] = ee.Geometry.MultiPolygon(subnational['geom_list'][i])

<ipython-input-24-7d0f26e5d259>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subnational['geom_ee'][i] = ee.Geometry.Polygon(subnational['geom_list'][i])
/Users/lavz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-24-7d0f26e5d259>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subnational['geom_ee'][i] =

In [25]:
## Getting health accessibility data
mean_h = []
med_h = []
std_h = []
hist_h = []
max_h = []
skew_h = []
pixel_h = []
subnational['health_access_mean'] = 1
subnational['health_access_median'] = 1
subnational['health_access_std'] = 1
subnational['health_access_skew'] = 1
subnational['health_access_hist'] = 1
subnational['health_access_max'] = 1
subnational['health_access_count'] = 1

In [27]:
for i in range(len(subnational)):
    try: 
        points_health = ee.Image('Oxford/MAP/accessibility_to_healthcare_2019').select('accessibility')
    
        mean_h.append(points_health.reduceRegion(**{ 
      'reducer': ee.Reducer.mean(),
      'geometry': subnational.geom_ee[i],
      'scale': 50,
      'maxPixels': 3e9}))
        subnational['health_access_mean'][i] = mean_h[i].getInfo()['accessibility']
    
    
        std_h.append(points_health.reduceRegion(**{
  'reducer': ee.Reducer.stdDev(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 3e9}))
        subnational['health_access_std'][i] = std_h[i].getInfo()['accessibility']
    
    
        med_h.append(points_health.reduceRegion(**{
  'reducer': ee.Reducer.median(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 3e9}))
        subnational['health_access_median'][i] = med_h[i].getInfo()['accessibility']
    
    
        skew_h.append(points_health.reduceRegion(**{
  'reducer': ee.Reducer.kurtosis(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 3e9}))
        subnational['health_access_skew'][i] = skew_h[i].getInfo()['accessibility']
    

    
        hist_h.append(points_health.reduceRegion(**{
  'reducer': ee.Reducer.fixedHistogram(0, 6000, 20),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 3e9}))
        subnational['health_access_hist'][i] = hist_h[i].getInfo()
    
    
        max_h.append(points_health.reduceRegion(**{
  'reducer': ee.Reducer.max(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 5e9}))
        subnational['health_access_max'][i] = max_h[i].getInfo()['accessibility']
        
        pixel_h.append(points_health.reduceRegion(**{
  'reducer': ee.Reducer.count(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 5e9}))
        subnational['health_access_count'][i] = pixel_h[i].getInfo()['accessibility']
        
        
        
    except HTTPError:
        continue
        
    except ee.EEException:
        continue

<ipython-input-27-b9a653db4db0>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subnational['health_access_mean'][i] = mean_h[i].getInfo()['accessibility']
<ipython-input-27-b9a653db4db0>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subnational['health_access_std'][i] = std_h[i].getInfo()['accessibility']
<ipython-input-27-b9a653db4db0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subnational['health_access_median'][i] = me

In [56]:
mean_ts = []
med_ts = []
std_ts = []
hist_ts = []
max_ts = []
skew_ts = []
pixel_ts = []
subnational['travelspeed_access_mean'] = 1.000
subnational['travelspeed_access_median'] = 1.000
subnational['travelspeed_access_std'] = 1.000
subnational['travelspeed_access_skew'] = 1.000
subnational['travelspeed_access_hist'] = 1.000
subnational['travelspeed_access_max'] = 1.000
subnational['travelspeed_access_count'] = 1.000

In [60]:
for i in range(len(subnational)):
    try: 
        points_speed = ee.Image('Oxford/MAP/friction_surface_2019').select('friction')
    
        mean_ts.append(points_speed.reduceRegion(**{
      'reducer': ee.Reducer.mean(),
      'geometry': subnational.geom_ee[i],
      'scale': 50,
      'maxPixels': 3e9}))
        subnational['travelspeed_access_mean'][i] = mean_ts[i].getInfo()['friction']
    
    
        std_ts.append(points_speed.reduceRegion(**{
  'reducer': ee.Reducer.stdDev(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 3e9}))
        subnational['travelspeed_access_std'][i] = std_ts[i].getInfo()['friction']
    
    
        med_ts.append(points_speed.reduceRegion(**{
  'reducer': ee.Reducer.median(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 3e9}))
        subnational['travelspeed_access_median'][i] = med_ts[i].getInfo()['friction']
    
    
        skew_ts.append(points_speed.reduceRegion(**{
  'reducer': ee.Reducer.kurtosis(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 3e9}))
        subnational['travelspeed_access_skew'][i] = skew_ts[i].getInfo()['friction']
    

    
        hist_ts.append(points_speed.reduceRegion(**{
  'reducer': ee.Reducer.fixedHistogram(0, 6000, 20),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 3e9}))
        subnational['travelspeed_access_hist'][i] = hist_ts[i].getInfo()
    
    
        max_ts.append(points_speed.reduceRegion(**{
  'reducer': ee.Reducer.max(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 5e9}))
        subnational['travelspeed_access_max'][i] = max_ts[i].getInfo()['friction']
        
        pixel_ts.append(points_speed.reduceRegion(**{
  'reducer': ee.Reducer.count(),
  'geometry': subnational.geom_ee[i],
  'scale': 50,
  'maxPixels': 5e9}))
        subnational['travelspeed_access_count'][i] = pixel_ts[i].getInfo()['friction']
        
        
        
    except HTTPError:
        continue
        
    except ee.EEException:
        continue

<ipython-input-60-840e6913d757>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subnational['travelspeed_access_mean'][i] = mean_ts[i].getInfo()['friction']
<ipython-input-60-840e6913d757>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subnational['travelspeed_access_std'][i] = std_ts[i].getInfo()['friction']
<ipython-input-60-840e6913d757>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subnational['travelspeed_access_median'][

In [62]:
subnational_infra2 = subnational.drop(['biomass_access_mean', 'biomass_access_median',
                        'biomass_access_std', 'biomass_access_skew', 'biomass_access_hist',
                            'biomass_access_max', 'biomass_access_count'], axis=1)

In [65]:
subnational_infra2.to_csv('subnational_health_friction.csv')  